# Python implementation of the substitution rules for the Penrose P3 tiling
## The substitution rules

The Penrose P3 tiling is made out of two rhombi: a thin and fat one.
Here we will work with half rhombi (triangles), as it makes use of the substitution rules easier. Thick edges mark the sides of the rhombi in the following figure.
<img src="http://preshing.com/images/red-blue-triangle.png">

The substituion rules for the triangles are as follows
<img src="http://preshing.com/images/red-triangle-subdivision.png">
<img src="http://preshing.com/images/blue-triangle-subdivision.png">
The new points $P$, $Q$ and $R$ separates the edges of the triangles into two parts whose ratio of lengthes is the golden ratio:
$$
 P = A + (B-A)\omega
$$
and
$$
 Q = B + (A-B)\omega \\
 R = B + (C-B)\omega
$$
where $\omega = (\sqrt{5}-1)/2$, the inverse of the golden ratio.

In [9]:
import cairo
import math, cmath

We shall describe a triangle by a color, and the data of its 3 vertices: (color, A, B, C).
With that in mind, we can easily implement the substition rules:

In [10]:
om = (math.sqrt(5) - 1.) / 2.

def subdivide(triangles):
    result = []
    for color, A, B, C in triangles:
        if color == 0:
            # Subdivide red triangle
            P = A + (B - A)*om
            result += [(0, C, P, B), (1, P, C, A)]
        else:
            # Subdivide blue triangle
            Q = B + (A - B)*om
            R = B + (C - B)*om
            result += [(1, R, C, A), (1, Q, R, B), (0, R, Q, A)]
    return result

In [11]:
lightred = (0.697, 0.425, 0.333)
lightblue = (0.333, 0.605, 0.697)
blue = (0., 0.4078, 0.5451)
red = (0.5451, 0.1373, 0.)

def draw(triangles):
    # Draw red triangles
    for color, A, B, C in triangles:
        if color == 0:
            cr.move_to(A.real, A.imag)
            cr.line_to(B.real, B.imag)
            cr.line_to(C.real, C.imag)
            cr.close_path()
    cr.set_source_rgb(0.697, 0.425, 0.333)
    cr.fill()    

    # Draw blue triangles
    for color, A, B, C in triangles:
        if color == 1:
            cr.move_to(A.real, A.imag)
            cr.line_to(B.real, B.imag)
            cr.line_to(C.real, C.imag)
            cr.close_path()
    cr.set_source_rgb(0., 0.4078, 0.5451)
    cr.fill()

    # Determine line width from size of first triangle
    color, A, B, C = triangles[0]
    cr.set_line_width(abs(B - A) / 10.0)
    cr.set_line_join(cairo.LINE_JOIN_ROUND)

    # Draw outlines
    for color, A, B, C in triangles:
        cr.move_to(C.real, C.imag)
        cr.line_to(A.real, A.imag)
        cr.line_to(B.real, B.imag)
        
    # set bg color
    cr.set_source_rgb(0.2, 0.2, 0.2)
    # commit to surface
    cr.stroke()

In [12]:
# Create wheel of red triangles around the origin
triangles = []
for i in range(10):
    B = cmath.rect(1, (2*i - 1) * math.pi / 10)
    C = cmath.rect(1, (2*i + 1) * math.pi / 10)
    if i % 2 == 0:
        B, C = C, B  # Make sure to mirror every second triangle
    triangles.append((0, 0j, B, C))

In [13]:
""" inside picture """

IMAGE_SIZE = (1600, 1600)

# Prepare cairo surface
surface = cairo.ImageSurface(cairo.FORMAT_ARGB32, IMAGE_SIZE[0], IMAGE_SIZE[1])
cr = cairo.Context(surface)
cr.translate(IMAGE_SIZE[0] / 2.0, IMAGE_SIZE[1] / 2.0) # centering
wheelRadius = 2.*((1./math.sqrt(2)) * math.sqrt((IMAGE_SIZE[0] / 2.0) ** 2 + (IMAGE_SIZE[1] / 2.0) ** 2))
cr.scale(wheelRadius, wheelRadius)

it_triangles = triangles
for i in range(6):
    it_triangles = subdivide(it_triangles)

draw(it_triangles)

surface.write_to_png("penrose.png")

In [7]:
""" cover image """
IMAGE_SIZE = (1600, 900)

# Prepare cairo surface
surface = cairo.ImageSurface(cairo.FORMAT_ARGB32, IMAGE_SIZE[0], IMAGE_SIZE[1])
cr = cairo.Context(surface)
cr.translate(IMAGE_SIZE[0] / 2.0, IMAGE_SIZE[1] / 2.0) # centering
wheelRadius = 2.*((1./math.sqrt(2)) * math.sqrt((IMAGE_SIZE[0] / 2.0) ** 2 + (IMAGE_SIZE[1] / 2.0) ** 2))
cr.scale(wheelRadius, wheelRadius)

In [8]:
it_triangles = triangles
for i in range(6):
    it_triangles = subdivide(it_triangles)

draw(it_triangles)

surface.write_to_png("cover.png")